In [1]:
import os
import pandas as pd
import numpy as np
import cv2
from glob import glob

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam


from sklearn.model_selection import train_test_split

In [2]:
def build_model(size, num_classes):
    inputs = Input((size, size, 3))
    back = MobileNetV2(input_tensor = inputs, include_top=False, weights="imagenet")
    back.trainable = True
    x = back.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    x = Dense(1024, activation="relu")(x)
    x = Dense(num_classes, activation="softmax")(x)
    
    model = tf.keras.Model(inputs, x)
    return model

In [3]:
def read_image(path, size):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (size,size))
    image = image / 255.0
    image = image.astype(np.float32)
    return image

In [4]:
def parse_data(x, y):
    x = x.decode()
    
    num_classes = 120
    size = 224
    
    image = read_image(x, size)
    label = [0] * num_classes
    label[y] = 1
    label = np.array(label)
    label = label.astype(np.int32)
    
    return image, label

In [5]:
def tf_parse(x, y):
    x, y = tf.numpy_function(parse_data, [x,y], [tf.float32, tf.int32])
    x.set_shape((224,224,3))
    y.set_shape((120))
    
    return x, y

In [6]:
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x,y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

In [7]:
path = "Dog_Breed_Ident/"
train_path = os.path.join(path, "train/*")
test_path = os.path.join(path, "test/*")
labels_path = os.path.join(path, "labels.csv")
    
labels_df = pd.read_csv(labels_path)
breed = labels_df["breed"].unique()
print("Number of Breed: ", len(breed))
    

Number of Breed:  120


In [8]:
breed2id = {name: i for i, name in enumerate(breed)}

ids = glob(train_path)
labels = []

for image_id in ids:
    image_id = image_id.split("/")[-1].split(".")[0]
    breed_name = list(labels_df[labels_df.id == image_id]["breed"])[0]
    breed_index = breed2id[breed_name]
    labels.append(breed_index)

ids = ids[:1000]
labels = labels[:1000]

In [9]:
train_x, valid_x = train_test_split(ids, test_size=0.2, random_state=42)
train_y, valid_y = train_test_split(labels, test_size=0.2, random_state=42)

In [16]:
#My Parameters
size = 224
num_classes = 120
rate = 1e-5
batch_size = 8
epochs = 10

In [11]:
model = build_model(size, num_classes)
model.compile(loss = "categorical_crossentropy", optimizer = Adam(rate), metrics=["acc"])

In [12]:
# Making Datasets
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

In [17]:
callbacks = [
    ModelCheckpoint("model.h5", verbose=1, save_best_only=True),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=1e-6)
]


train_steps = (len(train_x)//batch_size)+1
valid_steps = (len(valid_x)//batch_size)+1
model.fit(train_dataset, 
          steps_per_epoch=train_steps,
          validation_steps=valid_steps,
          validation_data=valid_dataset, 
          epochs=epochs, 
          callbacks=callbacks)

Epoch 1/10
101/101 [==============================] - 133s 1s/step - loss: 0.0105 - acc: 1.0000 - val_loss: 2.5055 - val_acc: 0.3700

Epoch 00001: val_loss improved from inf to 2.50551, saving model to model.h5
Epoch 2/10
101/101 [==============================] - 137s 1s/step - loss: 0.0084 - acc: 1.0000 - val_loss: 2.4873 - val_acc: 0.3850

Epoch 00002: val_loss improved from 2.50551 to 2.48732, saving model to model.h5
Epoch 3/10
101/101 [==============================] - 140s 1s/step - loss: 0.0068 - acc: 1.0000 - val_loss: 2.4548 - val_acc: 0.3950

Epoch 00003: val_loss improved from 2.48732 to 2.45481, saving model to model.h5
Epoch 4/10
101/101 [==============================] - 139s 1s/step - loss: 0.0057 - acc: 1.0000 - val_loss: 2.4332 - val_acc: 0.4200

Epoch 00004: val_loss improved from 2.45481 to 2.43324, saving model to model.h5
Epoch 5/10
101/101 [==============================] - 135s 1s/step - loss: 0.0046 - acc: 1.0000 - val_loss: 2.4082 - val_acc: 0.4250

Epoch 0000